# pyheatmy Demo Notebook

This notebook aims to present the various features of `pyheatmy`. It explains how to:
- create a `Column` object from an easy reading of dates
- execute the direct model, for homogeneous (section 2)  and heterogeneous (section 3) riverbeds
- execute the MCMC
- retrieve and display the various results produced during the executions of the direct model or the MCMC

This notebook doesn't provide yet information on the DREAM method implemented in 2023. For that purpose please refer to the `DREAM_VX.ipynb` notebooks

`pyheatmy` is built around the monolithic `Column` class in `core.py`. It can be executed from this class. Calculation, data retrieval, and plotting are methods provided by the `Column` class.

It is based on real data, which can be found in the `data` folder.

We recommend reading the API for more details. 

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
from pathlib import Path
from pyheatmy.core import (
    Column,
)
from pyheatmy.layers import Layer
from pyheatmy.params import Param, Prior
from pyheatmy.config import ZERO_CELSIUS, EPSILON

PROJECT_ROOT = Path('/Users/pgpetitmangin/molo/td/td3/molo/MOLONARI1D/pyheatmy')

## 1. Chargement des données et définition d'un objet ``Column``

On doit d'abord récupérer les données issues des capteurs, qui se trouvent dans le dossier ``data``.

On définit une fonction pour convertir les dates des dataframe, cela peut éviter certains problèmes.

In [ ]:
def convertDates(df: pd.DataFrame):
    """
    Convertit les dates d'un DataFrame pandas en testant plusieurs formats.
    """
    formats = (
        "%m/%d/%y %H:%M:%S",
        "%m/%d/%y %I:%M:%S %p",
        "%d/%m/%y %H:%M",
        "%d/%m/%y %I:%M %p",
        "%m/%d/%Y %H:%M:%S",
        "%m/%d/%Y %I:%M:%S %p",
        "%d/%m/%Y %H:%M",
        "%d/%m/%Y %I:%M %p",
        "%y/%m/%d %H:%M:%S",
        "%y/%m/%d %I:%M:%S %p",
        "%y/%m/%d %H:%M",
        "%y/%m/%d %I:%M %p",
        "%Y/%m/%d %H:%M:%S",
        "%Y/%m/%d %I:%M:%S %p",
        "%Y/%m/%d %H:%M",
        "%Y/%m/%d %I:%M %p",
        None,
    )
    times = df[df.columns[0]]
    for f in formats:
        try:
            new_times = pd.to_datetime(times, format=f)
            new_ts = new_times.values.astype(np.int64)
            test = np.sort(new_ts) - new_ts
            if np.sum(abs(test)) != 0:
                raise ValueError()
            df[df.columns[0]] = new_times
            return
        except ValueError:
            continue
    raise ValueError(
        "Impossible de convertir les dates: Aucun format connu ne correspond."
    )

On définit les objets ``dH_measures`` et ``T_measures``, nécessaires à la création d'un objet ``Column``, qui réalisera les calculs :
- ``dH_measures`` contient les dates des mesures, les mesures de différence de charge, et les températures de la rivière.
- ``T_measures`` contient les dates des mesures, et les températures mesurées par les 4 capteurs de la tige.

In [ ]:
# CONSTANTES DE SIMULATION
Zbottom = 0.4  # 40 cm
NBCELLS = 40
Zinterface = 0.2  # Interface entre les deux couches

# CHARGEMENT ET TRAITEMENT DES DONNÉES RÉELLES
print(
    "Chargement et traitement des données réelles"
)
try:
    capteur_riviere = pd.read_csv(
        PROJECT_ROOT / "data/Point034_processed/processed_pressures.csv",
        sep=",",
        names=["dates", "tension", "temperature_riviere"],
        skiprows=1,
    )
    capteur_ZH = pd.read_csv(
        PROJECT_ROOT / "data/Point034_processed/processed_temperatures.csv",
        sep=",",
        names=[
            "dates",
            "temperature_10",
            "temperature_20",
            "temperature_30",
            "temperature_40",
        ],
        skiprows=1,
    )
    etalonage_capteur_riv = pd.read_csv(
        PROJECT_ROOT / "configuration/pressure_sensors/P508.csv"
    )
except FileNotFoundError:
    print("ERREUR: Fichiers de données non trouvés.")
    exit()

# Traitement des dates
convertDates(capteur_riviere)
convertDates(capteur_ZH)

# Conversion de la pression en charge hydraulique (dH)
intercept = float(etalonage_capteur_riv["P508"][2])
a = float(etalonage_capteur_riv["P508"][3])
b = float(etalonage_capteur_riv["P508"][4])
capteur_riviere["dH"] = (
    capteur_riviere["tension"].astype(float)
    - intercept
    - capteur_riviere["temperature_riviere"].astype(float) * b
) / a

# Conversion des températures en Kelvin
capteur_riviere["temperature_riviere"] = (
    capteur_riviere["temperature_riviere"] + ZERO_CELSIUS
)
capteur_ZH["temperature_10"] = capteur_ZH["temperature_10"] + ZERO_CELSIUS
capteur_ZH["temperature_20"] = capteur_ZH["temperature_20"] + ZERO_CELSIUS
capteur_ZH["temperature_30"] = capteur_ZH["temperature_30"] + ZERO_CELSIUS
capteur_ZH["temperature_40"] = capteur_ZH["temperature_40"] + ZERO_CELSIUS

# Formatage des données pour la classe Column
dH_measures = list(
    zip(
        capteur_riviere["dates"],
        list(zip(capteur_riviere["dH"], capteur_riviere["temperature_riviere"])),
    )
)


T_measures = list(
    zip(
        capteur_ZH["dates"],
        capteur_ZH[
            ["temperature_10", "temperature_20", "temperature_30", "temperature_40"]
        ].to_numpy(),
    )
)

print("Données réelles chargées et formatées avec succès.")

On définit maintenant l'objet ``Column`` à partir d'un dictionnaire.

In [ ]:
# BLOC D'EXÉCUTION
if __name__ == "__main__":
    print("Création de l'instance de la classe Column avec les données réelles")

    col_dict = {
        "river_bed": Zbottom,
        "depth_sensors": [
            0.1,
            0.2,
            0.3,
            Zbottom,
        ],  # 10, 20, 30, 40cm 
        "offset": 0.0,
        "dH_measures": dH_measures,
        "T_measures": T_measures,
        "inter_mode": "lagrange",
        "nb_cells": NBCELLS,  # Utilise les 40 cellules
    }
    ma_colonne = Column.from_dict(col_dict)
    print("Instance créée.")

## 2. Multilayer Column

### 2.1. Direct Model

Pour une colonne stratifiée, on doit d'abord définir une liste d'objets ``Layer`` :

In [ ]:
    # Configuration du modèle multi-couches (2 couches)
print("Configuration du modèle multi-couches")
    
    
Couche_1_params = {
        "name": "Couche 1",
        "zLow": Zinterface,
        "IntrinK": 1e-11,  
        "n": 0.1,
        "lambda_s": 2,
        "rhos_cs": 4e6,
        "q_s": EPSILON,    # Terme source spécifique
    }
Couche_2_params = {
        "name": "Couche 2",
        "zLow": Zbottom,
        "IntrinK": 1e-11,
        "n": 0.1,
        "lambda_s": 2,
        "rhos_cs": 4e6,
        "q_s": EPSILON,
    }
    
Layer1 = Layer.from_dict(Couche_1_params)
Layer2 = Layer.from_dict(Couche_2_params)
ma_colonne.set_layers([Layer1, Layer2])
print("Modèle à 2 couches configuré.")

    # Configuration de la MCMC
print("Configuration de l'optimisation MCMC")
    
    
priors_couche_1 = {
        "Prior_IntrinK": ((1e-14, 1e-12), 0.01), 
        "Prior_n": ((0.01, 0.25), 0.01),
        "Prior_lambda_s": ((1, 10), 0.1),
        "Prior_rhos_cs": ((1e6, 1e7), 1e5),
        "Prior_q_s": ((-1e-5, 1e-5), 1e-10), # Prior simple autour de 0
    }
    
    # On utilise les mêmes plages de recherche pour la couche 2
priors_couche_2 = priors_couche_1.copy()

## 2.2 Inférence bayésienne

L'inférence bayésienne va nous permettre d'estimer une distribution a posteriori pour chaque paramètre.

### 2.2.1. MCMC sans estimation de l'erreur

On peut lancer une MCMC en gardant $\sigma^2$ constant. On définit des distributions a priori pour chaque couche :

In [ ]:
ma_colonne.all_layers[0].set_priors_from_dict(priors_couche_1)
ma_colonne.all_layers[1].set_priors_from_dict(priors_couche_2)
print("Priors assignés aux couches.")

        # Lancement de la MCMC
print("Lancement de la MCMC")
ma_colonne.compute_mcmc(
            verbose=True,
            nb_chain=10,  # 10 chaînes en parallèle
            nitmaxburning=250,  # Phase de chauffe
            nb_iter=1000,  # Itérations d'optimisation
        )
print("MCMC terminée")

#### Recupération et affichage des distributions

In [ ]:
#  Simulation Finale
print("Lancement de la Simulation Finale (Paramètres optimisés)")

        # Mise à jour de la colonne avec les meilleurs paramètres trouvés
ma_colonne.get_best_layers()
print("Meilleurs paramètres MCMC appliqués.")

In [ ]:
# Relance de la simulation avec les bons paramètres
ma_colonne.compute_solve_transi(verbose=False)
print("Calcul direct final terminé.")

In [ ]:
        # Affichage des résultats finaux
print("Affichage des résultats optimisés")
ma_colonne.plot_all_results()

In [ ]:
print("Affichage des distributions de paramètres trouvées")
ma_colonne.plot_all_param_pdf()

print("FIN DE LA SIMULATION COMPLÈTE")